<a href="https://colab.research.google.com/github/Se1towo/IMLO_Exam/blob/main/IMLO_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
flowers_102 = torchvision.datasets.Flowers102(
    root = "data",
    download=True)
print(flowers_102)

100%|██████████| 344862509/344862509 [00:16<00:00, 20496839.63it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 389626.32it/s]


100%|██████████| 14989/14989 [00:00<00:00, 19488041.74it/s]

Dataset Flowers102
    Number of datapoints: 1020
    Root location: data
    split=train
